# Smart Recommendation System for LoL Players


## Introduction

<img src="LoL.jpg">


The League of Legends (LoL) is a multiplayer online battle arena video game. Since its first release in 2009, the game has gained great popularity among players around the globe. In September 2016, it was estimated that there were over 100 million active players each month. From a larger scope, online gaming industry is consistently on the rise. Massively multiplayer(MMO) gaming generated revenue of roughly 19.9 billion U.S. dollars in 2016 [(1)](## References:). In this project, we obtains dynamic data including players and matches as well as static data from the official Riot API [(2)](## References:) through Cassiopeia, a framework dedicated to the Riot API [(3)](## References:). We then proceed onto further manipulation of the data and eventually building the champion recommendation system for LoL players.

## Table of Content

[1. Design and Implementation of Database](# Design and Loading of Database)

[2. Manipulation of Data](# Manipulation of Data)

[3. Prediction of Outcomes](# Prediction of Outcomes)

[4. Player Scoring System](# Player Scoring System)

[5. Champion Recommendation System](# Champion Recommendation System)

## Design and Implementation of  Database

In [14]:
from preprocess import *

## Manipulation of Data

## Prediction of Outcomes
With the data curated from the previous section, we build a binary classification model to predict the outcome of the match using support vector machine (SVM).

In [15]:
def predict_result(match_id, match_data, verbose=False):
    X = []
    y = []
    for i, match in enumerate(match_data):
        X.append([])
        # Avoid division by zero
        X[i].append(match['red']['total']['kda'] / (match['blue']['total']['kda'] + 1e-6))
        X[i].append(match['red']['total']['income'] / (match['blue']['total']['income'] + 1e-6))
        X[i].append(match['red']['total']['kills'] / (match['blue']['total']['kills'] + 1e-6))

        if match['win_side'] == 'red':
            y.append(0)
        else:
            y.append(1)
   
    scaler = MinMaxScaler()
    clf = svm.SVC(C=1e10, kernel='linear')
    X = np.array(X)
    y = np.array(y)
    scaler.fit(X)
    X_train = scaler.transform(X)
    clf.fit(X_train, y)
    if verbose:
        ret = clf.predict(X_train)
        tot = len(ret)
        hit = 0
        for pred, true in zip(ret, y):
            if pred == true:
                hit += 1
        print("train_accuracy:{:2.2f}%".format(hit * 100 / tot))
    
    ret = clf.predict(X_train[match_id].reshape([1, -1]))
    return 'red' if ret == 0 else 'blue', clf

ret, clf = predict_result(0, match_data, verbose=True)

print("Our prediction for match {} is WIN: {} and the true outcome is WIN: {}".format(0, ret, match_data[0]['win_side']))

train_accuracy:93.95%
Our prediction for match 0 is WIN: red and the true outcome is WIN: red


We can further infer which lane has the most impact on the outcome of the match using the model trained.

In [16]:
def which_lane(clf, match_data, verbose=False):
    
    lane_dict = {}
    for match in match_data:
        for lane in match['red']['by_lane'].keys():
            if lane not in match['blue']['by_lane'].keys():
                continue
            if lane not in lane_dict.keys():
                lane_dict[lane] = []
            
            # Avoid division by zero
            kda = match['red']['by_lane'][lane]['kda'] / (match['blue']['by_lane'][lane]['kda'] + 1e-6)
            income = match['red']['by_lane'][lane]['income'] / (match['blue']['by_lane'][lane]['income'] + 1e-6)
            kill = match['red']['by_lane'][lane]['kills'] / (match['blue']['by_lane'][lane]['kills'] + 1e-6)
            win = 0 if match['win_side'] == 'red' else 1
            lane_dict[lane].append([kda, income, kill, win])
    lanelist = [lane for lane in lane_dict.keys()]
    best_acc = 0.0
    the_lane = ''
    for lane, lane_data in lane_dict.items():
        if lane == None:
            continue
        lane_data = np.array(lane_data)
        X = lane_data[:,:3]
        y = lane_data[:,-1]
        scaler = MinMaxScaler()
        scaler.fit(X)
        X_train = scaler.transform(X)
        ret = clf.predict(X_train)
        tot = len(ret)
        hit = 0
        for pred, true in zip(ret, y):
            if pred == true:
                hit += 1
        acc = hit/tot
        if verbose:
            print("{} : {:2.2f}%".format(lane, acc * 100))
        if acc > best_acc:
            best_acc = acc
            the_lane = lane
    return the_lane

# Print our inference
lane = which_lane(clf, match_data)
print("{} has the most impact on the outcome of the match.".format(lane))

MID_LANE : 51.25%
JUNGLE : 51.30%
TOP_LANE : 53.20%
BOT_LANE : 51.29%
TOP_LANE has the most impact on the outcome of the match.


## Player Scoring System

## Champion Recommendation System

## Summary and Conclusion

## References:
[1]https://www.statista.com/topics/1551/online-gaming/

[2]https://developer.riotgames.com/

[3]https://cassiopeia.readthedocs.io/en/latest/